In [1]:
import pyopencl as cl
import numpy as np
import time, math, sys

mf = cl.mem_flags

In [2]:
platforms = cl.get_platforms()
cq = []
for platform in platforms:
    for dev in platform.get_devices():
        context = cl.Context(devices=[dev])
        queue = cl.CommandQueue(context=context)
        cq.append(( context, queue ))

cq

[(<pyopencl.Context at 0x55e7891cc010 on <pyopencl.Device 'Tesla K80' on 'NVIDIA CUDA' at 0x55e7891ddbe0>>,
  <pyopencl.cffi_cl.CommandQueue at 0x7f6de227afd0>),
 (<pyopencl.Context at 0x55e78a021e80 on <pyopencl.Device 'pthread-Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz' on 'Portable Computing Language' at 0x55e78952d950>>,
  <pyopencl.cffi_cl.CommandQueue at 0x7f6de2287090>)]

## create test harness and measure time

In [3]:
from IPython.display import display, HTML
def prettyprint(table):
    display(HTML('<table><tr>{0}</tr></table>'.format(
    "</tr><tr>".join( '<td>{}</td>'.format( 
        '</td><td>'.join(str(_) for _ in row)) for row in table )
    )))

In [4]:
def test(harness_builder = lambda context, queue, N: (lambda:None), count=10, N=1024):
    out = [ [ " " ] + [ cqu[0].devices[0].name for cqu in cq ], [ "Time (ms)"], [ "MFLOPS" ] ]
    for context, queue in cq:
        times = []
        harness, expected_reply = harness_builder(context, queue, N)
        print >> sys.stderr, "Testing with", context.devices[0].name
        result = None
        for i in xrange(count):
            time_start = time.time()
            try:
                result = harness()
            except Exception, e:
                print >> sys.stderr, "Exception on", context.devices[0].name, str(e)
                break
            times.append(time.time() - time_start)
            if expected_reply is not None:
                if ( (result-expected_reply) > 0.1 ).any():
                    print >> sys.stderr, "Warning! wrong result on", context.devices[0].name
            else:
                print >> sys.stderr, 'No expected reply'
        if times:
            out[1].append('{0:.3f}'.format( np.average(times)*1000 ) )
            out[2].append('{0:.0f}'.format( 2.0 * N * N * N/(1000000.0*np.average(times)) ) )
        else:
            out[1].append("N/A")
            out[2].append("N/A")

    prettyprint(out)
        

In [5]:
test(lambda a,b,c: (lambda: time.sleep(0.001), None), 10, 1024)

Testing with Tesla K80
No expected reply
No expected reply
No expected reply
No expected reply
No expected reply
No expected reply
No expected reply
No expected reply
No expected reply
No expected reply
Testing with pthread-Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
No expected reply
No expected reply
No expected reply
No expected reply
No expected reply
No expected reply
No expected reply
No expected reply
No expected reply
No expected reply


,Tesla K80,pthread-Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Time (ms),1.120,1.113
MFLOPS,1916711,1928613


In [6]:
prettyprint([ [ 'CPU', 'GPU' ], [1, 2]])

CPU,GPU
1,2


## view CPU/GPU capabilities

In [7]:
out = [ [ " " ] + [ cqu[0].devices[0].name for cqu in cq ] ]
out += [ [ "max work item size" ] + [ cqu[0].devices[0].max_work_item_sizes for cqu in cq ] ]
prettyprint(out)

,Tesla K80,pthread-Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
max work item size,"[1024L, 1024L, 64L]","[4096L, 4096L, 4096L]"


## run the program

In [ ]:
def harnessbuilder_basic(context, queue, N):
    h_A = np.random.rand(N,N).astype(np.float32)
    h_B = np.random.rand(N,N).astype(np.float32)
    h_C = np.empty([N,N]).astype(np.float32)
    expected_reply = h_A.dot(h_B)
    
    d_A = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_A)
    d_B = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_B)
    d_C = cl.Buffer(context, mf.WRITE_ONLY, h_C.nbytes)
    kernelsource_basic = """
       __kernel void mmul(__global const float* A,
                          __global const float* B,
                          __global float* out,
                          uint count)
        {
            __private uint i = get_global_id(0);
            __private uint j = get_global_id(1);
            __private uint k;
            __private float tmp = 0.0f;
            for (k=0; k<count; k++) {
                tmp += A[ i*count + k ] * B [ k*count + j ];
            }
            out[i*count + j] = tmp;
        }
    """
    program = cl.Program(context, kernelsource_basic).build()
    mmul = program.mmul
    mmul.set_scalar_arg_dtypes([None,None,None,np.uint32])

    def run_mmul():
        mmul(queue, h_A.shape, None, d_A, d_B, d_C, N)
        cl.enqueue_copy(queue, h_C, d_C)
        queue.finish()
        return h_C
    
    return run_mmul, expected_reply

In [ ]:
#test(harnessbuilder_basic, 10, 1024)

## version with row as work-item

In [ ]:
def harnessbuilder_row_as_workitem(context, queue, count):
    h_A = np.random.rand(count**2).astype(np.float32)
    h_B = np.random.rand(count**2).astype(np.float32)
    h_C = np.empty([count**2]).astype(np.float32)
    expected_reply = h_A.reshape(count,count).dot(h_B.reshape(count,count)).reshape(count**2)
    
    d_A = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_A)
    d_B = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_B)
    d_C = cl.Buffer(context, mf.WRITE_ONLY, h_C.nbytes)
    kernelsource = """
       __kernel void mmul(__global const float* A,
                          __global const float* B,
                          __global float* out,
                          uint count)
        {
            __private uint i = get_global_id(0);
            __private uint j, k;
            for (j=0; j<count; j++)
            {
                __private float tmp = 0.0f;
                for (k=0; k<count; k++) {
                    tmp += A[ i*count + k ] * B [ k*count + j ];
                }
                out[i*count + j] = tmp;
            }
        }
    """
    program = cl.Program(context, kernelsource).build()
    mmul = program.mmul
    mmul.set_scalar_arg_dtypes([None,None,None,np.uint32])

    def run_mmul():
        mmul(queue, (count,), (32,), d_A, d_B, d_C, count)
        cl.enqueue_copy(queue, h_C, d_C)
        queue.finish()
        return h_C
    
    return run_mmul, expected_reply

In [ ]:
#test(harnessbuilder_row_as_workitem, 10, 1024)

## version with row stored in private memory

In [ ]:
def harnessbuilder_row_cached_in_private(context, queue, count):
    h_A = np.random.rand(count**2).astype(np.float32)
    h_B = np.random.rand(count**2).astype(np.float32)
    h_C = np.empty([count**2]).astype(np.float32)
    expected_reply = h_A.reshape(count,count).dot(h_B.reshape(count,count)).reshape(count**2)
    
    d_A = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_A)
    d_B = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_B)
    d_C = cl.Buffer(context, mf.WRITE_ONLY, h_C.nbytes)
    kernelsource = """
       __kernel void mmul(__global const float* A,
                          __global const float* B,
                          __global float* out,
                          uint count)
        {
            __private uint i = get_global_id(0);
            __private uint j, k;
            __private float Arwk["""+str(count)+"""];
            
            for (k=0; k<count; k++)
                Arwk[k] = A[i*count+k];
            
            for (j=0; j<count; j++)
            {
                __private float tmp = 0.0f;
                for (k=0; k<count; k++) {
                    tmp += Arwk[k] * B [ k*count + j ];
                }
                out[i*count + j] = tmp;
            }
        }
    """
    program = cl.Program(context, kernelsource).build()
    mmul = program.mmul
    mmul.set_scalar_arg_dtypes([None,None,None,np.uint32])

    def run_mmul():
        mmul(queue, (count,), None, d_A, d_B, d_C, count)
        cl.enqueue_copy(queue, h_C, d_C)
        queue.finish()
        return h_C
    
    return run_mmul, expected_reply

In [ ]:
#test(harnessbuilder_row_cached_in_private, 10, 1024)

## version with local memory for the column

In [ ]:
def harnessbuilder_privateA_localB(context, queue, count):
    h_A = np.random.rand(count**2).astype(np.float32)
    h_B = np.random.rand(count**2).astype(np.float32)
    h_C = np.empty([count**2]).astype(np.float32)
    expected_reply = h_A.reshape(count,count).dot(h_B.reshape(count,count)).reshape(count**2)
    
    d_A = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_A)
    d_B = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_B)
    d_C = cl.Buffer(context, mf.WRITE_ONLY, h_C.nbytes)
    kernelsource = """
       __kernel void mmul(__global const float* A,
                          __global const float* B,
                          __global float* out,
                          __local float* Bwrk,
                          uint count)
        {
            __private uint i = get_global_id(0);
            __private uint iloc = get_local_id(0);
            __private uint nloc = get_local_size(0);
            __private uint j, k;
            __private float Arwk["""+str(count)+"""];
            
            for (k=0; k<count; k++)
                Arwk[k] = A[i*count+k];
            
            for (j=0; j < count; j++)
            {
                for( k = iloc; k<count; k+=nloc)
                {
                    Bwrk[k] = B[ k*count + j ];
                }
            
                barrier(CLK_LOCAL_MEM_FENCE);
                
                __private float tmp = 0.0f;
                for (k=0; k<count; k++) {
                    tmp += Arwk[k] * Bwrk[k];
                }
                out[i*count + j] = tmp;
                
                barrier(CLK_LOCAL_MEM_FENCE);
            }
                
        }
    """
    local = cl.LocalMemory(count*4)
    program = cl.Program(context, kernelsource).build()
    mmul = program.mmul
    mmul.set_scalar_arg_dtypes([None,None,None,None,np.uint32])

    def run_mmul():
        mmul(queue, (count,), None, d_A, d_B, d_C, local, count)
        cl.enqueue_copy(queue, h_C, d_C)
        queue.finish()
        return h_C
    
    return run_mmul, expected_reply

In [ ]:
#test(harnessbuilder_privateA_localB, 2, 1024)

## version split by blocks

In [ ]:
def harnessbuilder_blocks(context, queue, count):
    h_A = np.random.rand(count,count).astype(np.float32)
    h_B = np.random.rand(count,count).astype(np.float32)
    h_C = np.empty([count,count]).astype(np.float32)
    expected_reply = h_A.dot(h_B)
    
    d_A = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_A)
    d_B = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_B)
    d_C = cl.Buffer(context, mf.WRITE_ONLY, h_C.nbytes*4)
    
    BLOCK_SIZE = 16
    kernelsource = """

        #define BLOCK_SIZE """+str(BLOCK_SIZE)+"""
        //#define count """+str(count)+"""
        
        #define ASUB(i, j) Asub[i + j*BLOCK_SIZE]
        #define BSUB(i, j) Bsub[i + j*BLOCK_SIZE]



       __kernel void mmul(__global const float* A,
                          __global const float* B,
                          __global       float* out,
                          __local float* Asub,
                          __local float* Bsub,
                          __private uint count)
        {
        
            // this processing element has to compute out[ get_global_id(0), get_global_id(1) ]
            
            uint block_id_x = get_group_id(0);
            uint block_id_y = get_group_id(1);
            
            uint x_thread = get_local_id(0);
            uint y_thread = get_local_id(1);

            uint aStart = block_id_y * BLOCK_SIZE * count;
            uint aStep = BLOCK_SIZE;
            uint aEnd = aStart + count - 1;
            
            uint bStart = block_id_x * BLOCK_SIZE;
            uint bStep = BLOCK_SIZE * count;
            
            __private float Csub = 0.0f;
            
            for (uint a = aStart, b = bStart;
                 a <= aEnd;
                 a += aStep, b+= bStep)
                 {
                     ASUB(x_thread, y_thread) = A[a + count * y_thread + x_thread];
                     BSUB(x_thread, y_thread) = B[b + count * y_thread + x_thread];
                     
                     barrier(CLK_LOCAL_MEM_FENCE);
                     
                     #pragma unroll
                     for( uint k =0; k < BLOCK_SIZE; ++k)
                         Csub += ASUB(k, y_thread) * BSUB(x_thread, k);
                         
                     barrier(CLK_LOCAL_MEM_FENCE);
                 }
                 
            out[get_global_id(1) * count + get_global_id(0)] = Csub;
                
        }
    """
    local_a = cl.LocalMemory(4*(BLOCK_SIZE**2))
    local_b = cl.LocalMemory(4*(BLOCK_SIZE**2))
    program = cl.Program(context, kernelsource).build()
    mmul = program.mmul
    mmul.set_scalar_arg_dtypes([None,None,None,None,None,np.uint32])

    def run_mmul():
        mmul(queue, (count,count), (BLOCK_SIZE,BLOCK_SIZE), d_A, d_B, d_C, local_a, local_b, count)
        cl.enqueue_copy(queue, h_C, d_C)
        queue.finish()
        return h_C
    
    return run_mmul, expected_reply

In [ ]:
test(harnessbuilder_blocks, 1, 1024*4)

## version split by blocks - Integer!

In [10]:
def harnessbuilder_blocks_int(context, queue, count):
    h_A = np.random.rand(count,count).astype(np.int32)
    h_B = np.random.rand(count,count).astype(np.int32)
    h_C = np.empty([count,count]).astype(np.int32)
    expected_reply = h_A.dot(h_B)
    
    d_A = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_A)
    d_B = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_B)
    d_C = cl.Buffer(context, mf.WRITE_ONLY, h_C.nbytes*4)
    
    BLOCK_SIZE = 16
    kernelsource = """

        #define BLOCK_SIZE """+str(BLOCK_SIZE)+"""
        //#define count """+str(count)+"""
        
        #define ASUB(i, j) Asub[i + j*BLOCK_SIZE]
        #define BSUB(i, j) Bsub[i + j*BLOCK_SIZE]



       __kernel void mmul(__global const int* A,
                          __global const int* B,
                          __global       int* out,
                          __local int* Asub,
                          __local int* Bsub,
                          __private uint count)
        {
        
            // this processing element has to compute out[ get_global_id(0), get_global_id(1) ]
            
            uint block_id_x = get_group_id(0);
            uint block_id_y = get_group_id(1);
            
            uint x_thread = get_local_id(0);
            uint y_thread = get_local_id(1);

            uint aStart = block_id_y * BLOCK_SIZE * count;
            uint aStep = BLOCK_SIZE;
            uint aEnd = aStart + count - 1;
            
            uint bStart = block_id_x * BLOCK_SIZE;
            uint bStep = BLOCK_SIZE * count;
            
            __private int Csub = 0;
            
            for (uint a = aStart, b = bStart;
                 a <= aEnd;
                 a += aStep, b+= bStep)
                 {
                     ASUB(x_thread, y_thread) = A[a + count * y_thread + x_thread];
                     BSUB(x_thread, y_thread) = B[b + count * y_thread + x_thread];
                     
                     barrier(CLK_LOCAL_MEM_FENCE);
                     
                     #pragma unroll
                     for( uint k =0; k < BLOCK_SIZE; ++k)
                         Csub += ASUB(k, y_thread) * BSUB(x_thread, k);
                         
                     barrier(CLK_LOCAL_MEM_FENCE);
                 }
                 
            out[get_global_id(1) * count + get_global_id(0)] = Csub;
                
        }
    """
    local_a = cl.LocalMemory(4*(BLOCK_SIZE**2))
    local_b = cl.LocalMemory(4*(BLOCK_SIZE**2))
    program = cl.Program(context, kernelsource).build()
    mmul = program.mmul
    mmul.set_scalar_arg_dtypes([None,None,None,None,None,np.uint32])

    def run_mmul():
        mmul(queue, (count,count), (BLOCK_SIZE,BLOCK_SIZE), d_A, d_B, d_C, local_a, local_b, count)
        cl.enqueue_copy(queue, h_C, d_C)
        queue.finish()
        return h_C
    
    return run_mmul, expected_reply

In [12]:
test(harnessbuilder_blocks_int, 1, 1024*2)

Testing with Tesla K80
Testing with pthread-Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz


,Tesla K80,pthread-Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Time (ms),120.566,10197.357
MFLOPS,142493,1685
